In [1]:
import pulp

# 2.1

In [2]:
problem = pulp.LpProblem('SLE', pulp.LpMaximize)
# SLE= Simultaneous Liner Equations

x = pulp.LpVariable('x', cat='Continuous') # 連続変数
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y ==1440
problem += x + y == 10

status = problem.solve()

In [3]:
'Status:', pulp.LpStatus[status]

('Status:', 'Optimal')

In [4]:
f'x={x.value()}, y={y.value()}'

'x=2.0, y=8.0'

# 2.2

- Product p -> material m = 1kg/kg, n = 2kg/kg
- Product q -> material m = 3kg/kg, n = 1kg/kg
- material m <= 30, n <= 40
- profit p=1, q=2

In [5]:
problem = pulp.LpProblem('product', pulp.LpMaximize)

x1 = pulp.LpVariable('x1', cat='Continuous')
x2 = pulp.LpVariable('x2', cat='Continuous')

problem += x1 + 2 * x2

problem += x1 + 3 * x2 <= 30
problem += 2 * x1 + x2 <= 40

problem += x1 >= 0
problem += x2 >= 0

status = problem.solve()

In [6]:
'Status:', pulp.LpStatus[status]

('Status:', 'Optimal')

In [7]:
f'p={x1.value()}, q={x2.value()}, obj={problem.objective.value()}'

'p=18.0, q=4.0, obj=26.0'

# 2.3

In [8]:
import pandas as pd

In [11]:
df_stock = pd.read_csv('stocks.csv')
df_requires = pd.read_csv('requires.csv')
df_gains = pd.read_csv('gains.csv')

In [26]:
P = df_gains['p'].tolist()
M = df_stock['m'].to_list()

In [25]:
stock = df_stock.set_index('m')['stock'].to_dict()
requires = df_requires.set_index(['p', 'm'])['require'].to_dict()
gain = df_gains.set_index('p')['gain'].to_dict()

In [46]:
def change_variable_type(type_name):
    result =''

    problem = pulp.LpProblem('LP2', pulp.LpMaximize)
    x = pulp.LpVariable.dicts('x', P, cat=type_name)

    for p in P:
        problem += x[p] >= 0

    for m in M:
        problem += pulp.lpSum([requires[p,m] * x[p] for p in P]) <= stock[m]


    problem += pulp.lpSum([gain[p] * x[p] for p in P])
    status = problem.solve()
    result += f'Status: {pulp.LpStatus[status]}'
    result += '\n'

    for p in P:
        result += f'{p}, {x[p].value()}'
        result += '\n'
    result += f'obj={problem.objective.value()}'
    return result

In [47]:
print(change_variable_type('Continuous'))

Status: Optimal
p1, 12.142857
p2, 3.5714286
p3, 7.4285714
p4, 0.0
obj=80.42857099999999


In [48]:
print(change_variable_type('Integer'))

Status: Optimal
p1, 13.0
p2, 3.0
p3, 7.0
p4, -0.0
obj=79.0
